In [5]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix
 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 32

Creamos Set de Entrenamiento

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(os.getcwd(), 'CarneDataset/train'),seed =123,image_size=(IMG_SIZE,IMG_SIZE))
classes_train = train_ds.class_names
nClasses_train = len(classes_train)

Found 1633 files belonging to 8 classes.



Creamos el modelo de CNN y Configuramos los parametros de la red neuronal

In [7]:
epochs = 30 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 64 # Cantidad de imágenes que se toman a la vez en memoria

classifier_model = Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape = (IMG_SIZE,IMG_SIZE,3)),
    Conv2D(16,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(32,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(128,activation='relu'),
    Dense(nClasses_train)
])
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
classifier_model.compile(optimizer=optimizer, 
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=["accuracy"])

Entrenamos el modelo CNN

In [8]:
classifier_train = classifier_model.fit(train_ds,batch_size=batch_size,epochs=epochs)

Epoch 1/30


2023-06-30 17:45:26.369431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1633]
	 [[{{node Placeholder/_0}}]]
2023-06-30 17:45:26.370077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [1633]
	 [[{{node Placeholder/_4}}]]


52/52 [==============================] - 3s 44ms/step - loss: 1.4075 - accuracy: 0.5713
Epoch 2/30
52/52 [==============================] - 2s 41ms/step - loss: 1.2174 - accuracy: 0.5903
Epoch 3/30
52/52 [==============================] - 2s 42ms/step - loss: 0.9015 - accuracy: 0.6816
Epoch 4/30
52/52 [==============================] - 3s 50ms/step - loss: 0.6664 - accuracy: 0.7446
Epoch 5/30
52/52 [==============================] - 3s 51ms/step - loss: 0.5447 - accuracy: 0.7924
Epoch 6/30
52/52 [==============================] - 3s 46ms/step - loss: 0.4778 - accuracy: 0.8126
Epoch 7/30
52/52 [==============================] - 2s 39ms/step - loss: 0.4268 - accuracy: 0.8359
Epoch 8/30
52/52 [==============================] - 3s 47ms/step - loss: 0.4766 - accuracy: 0.8200
Epoch 9/30
52/52 [==============================] - 3s 47ms/step - loss: 0.3934 - accuracy: 0.8481
Epoch 10/30
52/52 [==============================] - 3s 47ms/step - loss: 0.3648 - accuracy: 0.8751
Epoch 11/30
52/52 [=

Predicción para Test

In [10]:
datagen = ImageDataGenerator()
test_generator = datagen.flow_from_directory(os.path.join(os.getcwd(), 'CarneDataset/test'),
                                            target_size = (IMG_SIZE,IMG_SIZE),class_mode = 'categorical',shuffle=False)



predicted_test = classifier_model.predict(test_generator)

y_predict_test = np.argmax(predicted_test,axis=1)
y_real_test = test_generator.classes
correct_test = np.where(y_predict_test==y_real_test)[0]

print ('Correct test values %s from %s'%(len(correct_test),len(y_real_test)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_test)/len(y_real_test)*100))
confusion_matrix(y_real_test, y_predict_test)

Found 810 images belonging to 8 classes.


 1/26 [>.............................] - ETA: 4s

2023-06-30 17:48:29.674304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


26/26 [==============================] - 3s 125ms/step
Correct test values 733 from 810
Porcentaje valores correctos: 90.49


array([[  0,   0,   0,   0,   0,   0,   1,   0],
       [  0,  39,   0,   9,   0,   0,   0,   0],
       [  0,   0,  75,   0,   2,   0,  20,   0],
       [  0,   2,   0,  42,   0,   1,   0,   0],
       [  0,   1,   5,   0, 442,   0,   4,   7],
       [  0,   0,   0,   0,   4,  15,   0,   0],
       [  0,   0,   8,   2,   4,   0, 100,   0],
       [  0,   0,   0,   0,   7,   0,   0,  20]])

Predicción para Training

In [12]:
train_generator = datagen.flow_from_directory(os.path.join(os.getcwd(), 'CarneDataset/train'),
                                            target_size = (IMG_SIZE,IMG_SIZE),class_mode = 'categorical',shuffle=False)

predicted_train = classifier_model.predict(train_generator)
y_predict_train = np.argmax(predicted_train,axis=1)
y_real_train = train_generator.classes
correct_train = np.where(y_predict_train==y_real_train)[0]

print ('Correct train values %s from %s'%(len(correct_train),len(y_real_train)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_train)/len(y_real_train)*100))
confusion_matrix(y_real_train, y_predict_train)

Found 1633 images belonging to 8 classes.
 1/52 [..............................] - ETA: 8s

2023-06-30 17:50:10.735800: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


52/52 [==============================] - 7s 127ms/step
Correct train values 1604 from 1633
Porcentaje valores correctos: 98.22


array([[ 60,   0,   2,   0,   0,   0,   0],
       [  0, 197,   0,   3,   0,  13,   0],
       [  0,   0, 105,   0,   0,   0,   0],
       [  0,   4,   0, 941,   0,   3,   1],
       [  0,   0,   0,   1,  36,   0,   0],
       [  1,   0,   1,   0,   0, 202,   0],
       [  0,   0,   0,   0,   0,   0,  63]])